In [1]:
import os
from PIL import Image, ImageFilter
import numpy as np

In [2]:
def get_image_size(image_path):
    with Image.open(image_path) as img:
        return img.size

In [3]:
def check_image_sizes(directory, output_file):
    image_sizes = {}
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
                file_path = os.path.join(root, file)
                try:
                    size = get_image_size(file_path)
                    if size in image_sizes:
                        image_sizes[size].append(file_path)
                    else:
                        image_sizes[size] = [file_path]
                except Exception as e:
                    print(f"Could not process image {file_path}: {e}", file=output_file)

    if len(image_sizes) == 1:
        print("All images are the same size.", file=output_file)
    else:
        print("Images have different sizes.", file=output_file)
        for size, paths in image_sizes.items():
            print(f"Size {size}: {len(paths)} images", file=output_file)

In [4]:
# Info text file on the different image sizes in the raw dataset
directory = 'dataset5/'
output_file_path = 'imagesizes.txt'

with open(output_file_path, 'w') as output_file:
    check_image_sizes(directory, output_file)

print(f"Results written to {output_file_path}")

Results written to imagesizes.txt


In [5]:
def apply_custom_sharpen(image):
    """
    Applies a custom sharpening filter to an image - filter taken from paper

    Args:
        image (Image): The input image.

    Returns:
        Image: The sharpened image.
    """
    # Define a 3x3 sharpening kernel
    kernel = [
        0, -1, 0,
        -1, 6, -1,
        0, -1, 0
    ]
    # Apply the kernel to the image
    sharpened_image = image.filter(ImageFilter.Kernel((3, 3), kernel, scale=1, offset=0))
    return sharpened_image

In [6]:
def preprocess_image(image_path):
    """
    Preprocesses an image by converting it to grayscale, applying a custom sharpening filter,
    resizing it to 50x50 pixels, and normalizing the pixel values to the range [0, 1].

    Args:
        image_path (str): Path to the input image.

    Returns:
        Image: The preprocessed image.
    """
    # Open image and convert to grayscale
    image = Image.open(image_path).convert('L')
    
    # Resize the image to 50x50 pixels
    resized_image = image.resize((50, 50))
    
    # Apply the custom sharpening filter
    sharpened_image = apply_custom_sharpen(resized_image)
    
    # Normalize the pixel values to the range [0, 1]
    normalized_image = np.array(sharpened_image) / 255.0
    
    # Convert back to an image format
    return Image.fromarray((normalized_image * 255).astype(np.uint8))

In [7]:
def process_directory(input_dir, output_dir):
    """
    Processes all color images in the specified directory and its subdirectories,
    applying the preprocessing steps and saving the results to the output directory,
    maintaining the original directory structure.

    Args:
        input_dir (str): Path to the input directory containing images.
        output_dir (str): Path to the output directory where preprocessed images will be saved.
    """
    for root, _, files in os.walk(input_dir):
        for file in files:
            # Process only color images with specified file extensions
            if file.lower().startswith('color') and file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
                file_path = os.path.join(root, file)
                
                # Determine the new file path in the output directory
                relative_path = os.path.relpath(root, input_dir)
                output_path = os.path.join(output_dir, relative_path, file)
                
                # Ensure the output directory exists
                os.makedirs(os.path.dirname(output_path), exist_ok=True)
                
                # Preprocess and save the image
                try:
                    processed_image = preprocess_image(file_path)
                    processed_image.save(output_path)
                except Exception as e:
                    print(f"Could not process image {file_path}: {e}")

In [8]:
input_directory = 'dataset5'
output_directory = 'dataset5_preprocessed_50x50'
process_directory(input_directory, output_directory)
print("hehehe finished")